In [1]:
import os
import json
import csv
import wave
import cv2
import numpy as np
import pandas as pd
import ffmpeg
import joblib
import tensorflow as tf
import mediapipe as mp
import torch  # For YOLO object detection
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import InputLayer as KerasInputLayer  # For custom layer
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector, ThresholdDetector

import vosk
import torch
import torch.nn as nn
import os
import json
import csv
import wave
import cv2
import numpy as np
import pandas as pd
import ffmpeg
import joblib
import tensorflow as tf
import mediapipe as mp
import torch  # For YOLO object detection
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import InputLayer as KerasInputLayer  # For custom layer


ModuleNotFoundError: No module named 'ffmpeg'

In [ ]:


vgg_face_descriptor = load_model('../models/vgg_face_descriptor.h5')
scaler = joblib.load('../models/scaler.pkl')
pca = joblib.load('../models/pca.pkl')
clf = joblib.load('../models/svm_classifier.pkl')
le = joblib.load('../models/label_encoder.pkl')
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

# -------------------- Action Detection Function --------------------






NameError: name 'nn' is not defined

In [ ]:
# -------------------- Pipeline Functions --------------------
# Function 1: Detect Scenes using scenedetect
def detect_scenes(video_path, content_threshold=50.0, threshold_val=20, min_scene_len=15):
    from scenedetect import open_video, SceneManager
    from scenedetect.detectors import ContentDetector, ThresholdDetector

    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=content_threshold))
    scene_manager.add_detector(ThresholdDetector(threshold=threshold_val, min_scene_len=min_scene_len))
    scene_manager.detect_scenes(video=video)
    scene_list = scene_manager.get_scene_list()
    print(f"[LOG] detect_scenes: Detected {len(scene_list)} scene(s) in '{video_path}'.")
    return scene_list

# Function 2: Trim Video Clip using ffmpeg-python
def trim_clip(input_video, start_time, end_time, clip_index, max_duration=4.0, output_dir='clips'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    duration = end_time - start_time
    if duration > max_duration:
        end_time = start_time + max_duration  # Truncate to max_duration
    clip_name = f"uploaded_video_{clip_index:03d}.mp4"
    output_path = os.path.join(output_dir, clip_name)
    
    try:
        ffmpeg.input(input_video, ss=start_time, t=(end_time - start_time)) \
              .output(output_path, vcodec='libx264', acodec='aac') \
              .run(overwrite_output=True)
    except ffmpeg.Error as e:
        print("Error trimming clip:", e)
    
    print(f"[LOG] trim_clip: Trimmed clip {clip_name} from {start_time}s to {end_time}s (max {max_duration}s).")
    return output_path

# Function 3: Extract Key Frame using OpenCV
def extract_key_frame(clip_path, frame_index='middle', output_dir='frames'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    cap = cv2.VideoCapture(clip_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps if fps > 0 else 0
    
    # Determine target time: use the middle of the clip by default
    target_time = duration / 2 if frame_index == 'middle' else 0
    target_frame = int(target_time * fps)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)
    ret, frame = cap.read()
    if ret:
        frame_file_name = os.path.basename(clip_path).split('.')[0] + "_keyframe.jpg"
        frame_path = os.path.join(output_dir, frame_file_name)
        cv2.imwrite(frame_path, frame)
        print(f"[LOG] extract_key_frame: Extracted key frame at {target_time:.2f}s -> {frame_path}")
        cap.release()
        return frame_path
    else:
        cap.release()
        print("[LOG] extract_key_frame: Failed to extract frame.")
        return None

# Function 4: Speech-to-Text using Vosk (with ffmpeg for audio extraction)
def speech_to_text(clip_path, model_path="model"):
    temp_audio = "temp_audio.wav"
    
    try:
        ffmpeg.input(clip_path).output(temp_audio, ac=1, ar='16k').run(overwrite_output=True)
    except ffmpeg.Error as e:
        print("Error extracting audio:", e)
        return ""
    
    wf = wave.open(temp_audio, "rb")
    print("[LOG] speech_to_text: Checking audio format...")
    print("    Channels:", wf.getnchannels())
    print("    Sample Width:", wf.getsampwidth())
    print("    Frame Rate:", wf.getframerate())
    
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
        wf.close()
        os.remove(temp_audio)
        raise ValueError("Audio must be WAV mono PCM at 16kHz.")
    
    import vosk
    model_instance = vosk.Model(lang="en-us") if model_path == "model" else vosk.Model(model_path)
    rec = vosk.KaldiRecognizer(model_instance, wf.getframerate())
    
    results = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result_json = json.loads(rec.Result())
            results.append(result_json)
    
    final_result = json.loads(rec.FinalResult())
    results.append(final_result)
    wf.close()
    os.remove(temp_audio)
    
    recognized_texts = [r["text"] for r in results if "text" in r]
    full_text = " ".join(recognized_texts)
    print(f"[LOG] speech_to_text: Transcribed {len(full_text.split())} word(s).")
    return full_text

# Function 5: Face Detection & Recognition using MediaPipe
def recognize_face_mediapipe(frame_path):
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
    
    image = cv2.imread(frame_path)
    if image is None:
        print("[LOG] recognize_face_mediapipe: Could not read image from", frame_path)
        return "NoFaceDetected"
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)
    
    if not results.detections:
        print("[LOG] recognize_face_mediapipe: No face detected in the frame.")
        return "NoFaceDetected"
    
    detection = results.detections[0]
    bboxC = detection.location_data.relative_bounding_box
    ih, iw, _ = image.shape
    x_min = max(0, int(bboxC.xmin * iw))
    y_min = max(0, int(bboxC.ymin * ih))
    width = int(bboxC.width * iw)
    height = int(bboxC.height * ih)
    face_snippet = image[y_min:y_min+height, x_min:x_min+width]
    
    identity = recognize_face_from_snippet(face_snippet)
    return identity

def recognize_face_from_snippet(face_snippet):
    global vgg_face_descriptor, scaler, pca, clf, le
    img = cv2.cvtColor(face_snippet, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    img_expanded = np.expand_dims(img, axis=0)
    
    embedding_vector = vgg_face_descriptor.predict(img_expanded)[0]
    embedding_vector = scaler.transform([embedding_vector])
    embedding_vector = pca.transform(embedding_vector)
    y_pred = clf.predict(embedding_vector)
    predicted_name = le.inverse_transform(y_pred)[0]
    return predicted_name

# Function 6: Object Detection using YOLOv5
def detect_objects_yolo(image_path, conf_threshold=0.5):
    image = cv2.imread(image_path)
    if image is None:
        print(f"[LOG] detect_objects_yolo: Could not read image from {image_path}")
        return []
    
    results = yolo_model(image)
    df = results.pandas().xyxy[0]
    df = df[df['confidence'] >= conf_threshold]
    
    img_area = image.shape[0] * image.shape[1]
    detections = []
    for index, row in df.iterrows():
        x1, y1, x2, y2 = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        bbox_area = (x2 - x1) * (y2 - y1)
        prominence = bbox_area / img_area
        detections.append({
            "class": row['name'],
            "confidence": float(row['confidence']),
            "prominence": prominence
        })
    
    print(f"[LOG] detect_objects_yolo: Found {len(detections)} high-confidence objects.")
    return detections

def recognize_text(image_path):
    try:
        # Open the image using PIL and run pytesseract OCR
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        print(f"[LOG] recognize_text: OCR found {len(text.split())} words.")
        return text.strip()
    except Exception as e:
        print("[LOG] recognize_text: Error processing OCR:", e)
        return ""
    
                           


In [ ]:


def create_combined_text(record):
    """Combine key fields into one searchable text field."""
    parts = []
    # Include face detection if present and not "NoFaceDetected"
    face = record.get("face_detected", "")
    if face and face.lower() != "nofacedetected":
        parts.append(face.replace("pins_", "").lower())
    # Include caption if available
    caption = record.get("caption", "")
    if caption:
        parts.append(caption.lower())
    # Parse object_detection JSON and add object classes
    try:
        objects = json.loads(record.get("object_detection", "[]"))
        obj_names = [obj.get("class", "").lower() for obj in objects if "class" in obj]
        parts.extend(obj_names)
    except Exception as e:
        pass
    # Include OCR text if available
    ocr = record.get("ocr_text", "")
    if ocr:
        parts.append(ocr.lower())
    # Include action detection if not "Unknown"
    action = record.get("action_detected", "")
    if action and action.lower() != "unknown":
        parts.append(action.lower())
    return " ".join(parts)

# -------------------- Main Processing Pipeline --------------------
def process_video(video_path):
    print(f"[LOG] process_video: Starting processing for '{video_path}'...")
    
    # 1) Scene Detection
    scenes = detect_scenes(video_path, content_threshold=10.0, threshold_val=20, min_scene_len=15)
    print(f"[DEBUG] process_video: Scenes -> {scenes}")
    if not scenes:
        print("[DEBUG] process_video: No scenes detected! The CSV will likely be empty.")
    
    results = []
    # Ensure output directories exist (for clips and frames in ../input)
    clips_dir = os.path.join("..", "input", "clips")
    frames_dir = os.path.join("..", "input", "frames")
    os.makedirs(clips_dir, exist_ok=True)
    os.makedirs(frames_dir, exist_ok=True)
    
    for idx, scene in enumerate(scenes, start=1):
        start_sec = scene[0].get_seconds()
        end_sec = scene[1].get_seconds()
        print(f"[LOG] process_video: Processing scene #{idx} | Start: {start_sec:.2f}s, End: {end_sec:.2f}s.")
        
        # 2) Trim the scene, output to ../input/clips
        clip_path = trim_clip(video_path, start_sec, end_sec, idx, max_duration=4.0, output_dir=clips_dir)
        
        # 3) Extract key frame, output to ../input/frames
        frame_path = extract_key_frame(clip_path, frame_index='middle', output_dir=frames_dir)
        
        # 4) Speech-to-Text from the clip
        recognized_speech = speech_to_text(clip_path)
        
        # 5) Face detection & recognition on the key frame
        face_identity = recognize_face_mediapipe(frame_path)
        
        # 6) Object detection on the key frame
        object_detections = detect_objects_yolo(frame_path)
        
        # 7) Action detection on the key frame
    
        
        # 8) OCR to extract text
        ocr_text = recognize_text(frame_path)
        
        # Collect results for this scene
        record = {
            "clip_name": os.path.basename(clip_path),
            "frame_name": os.path.basename(frame_path) if frame_path else "None",
            "face_detected": face_identity,
            "caption": recognized_speech,
            "object_detection": json.dumps(object_detections),
            "ocr_text": ocr_text,
            #add action detected here
            "timestamp_start": start_sec,
            "timestamp_end": end_sec
        }
        # Create the combined text for NLP search
        record["combined_text"] = create_combined_text(record)
        
        results.append(record)
        print(f"[LOG] process_video: Scene #{idx} processing complete.\n")
    
    print(f"[DEBUG] process_video: Final results -> {results}")
    
    # Create output directory for CSV
    csv_dir = os.path.join("..", "output", "csv")
    os.makedirs(csv_dir, exist_ok=True)
    csv_filename = os.path.join(csv_dir, "final_results.csv")
    
    # Write results to CSV, including the combined_text column
    fieldnames = [
        "clip_name", "frame_name", "face_detected", "caption",
        "object_detection", "ocr_text", "action_detected",
        "timestamp_start", "timestamp_end", "combined_text"
    ]
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
    
    print(f"[LOG] process_video: All scenes processed. Results saved to '{csv_filename}'.")
    print("Final Aggregated Results:")
    for record in results:
        print(record)

# -------------------- Main Guard --------------------
if __name__ == "__main__":
    print("[DEBUG] Starting main pipeline process...")
    input_video = "../input/video.mp4"
    process_video(input_video)


[DEBUG] Starting main pipeline process...
[LOG] process_video: Starting processing for '../input/video.mp4'...
[LOG] detect_scenes: Detected 83 scene(s) in '../input/video.mp4'.
[DEBUG] process_video: Scenes -> [(00:00:00.000 [frame=0, fps=30.000], 00:00:03.133 [frame=94, fps=30.000]), (00:00:03.133 [frame=94, fps=30.000], 00:00:05.533 [frame=166, fps=30.000]), (00:00:05.533 [frame=166, fps=30.000], 00:00:07.633 [frame=229, fps=30.000]), (00:00:07.633 [frame=229, fps=30.000], 00:00:08.333 [frame=250, fps=30.000]), (00:00:08.333 [frame=250, fps=30.000], 00:00:10.633 [frame=319, fps=30.000]), (00:00:10.633 [frame=319, fps=30.000], 00:00:13.833 [frame=415, fps=30.000]), (00:00:13.833 [frame=415, fps=30.000], 00:00:19.400 [frame=582, fps=30.000]), (00:00:19.400 [frame=582, fps=30.000], 00:00:22.933 [frame=688, fps=30.000]), (00:00:22.933 [frame=688, fps=30.000], 00:00:26.433 [frame=793, fps=30.000]), (00:00:26.433 [frame=793, fps=30.000], 00:00:30.833 [frame=925, fps=30.000]), (00:00:30.83

NameError: name 'mp' is not defined